## Programming Drill 3.3.2 
### Modify your program from Programming drill 3.2.2 so that you allow transitions from the many slits to the many measuring devices to be complex numbers. Your program should identify where there are interference phenomena. 

How interference is detected here

Compute the amplitude distribution at the targets when all slits are open (matrix B built from the full gun→slits amplitudes). Produce probabilities P_all = |ψ_all|².

For each slit j, create a modified matrix B_j where the gun only sends amplitude to slit j (every other gun→slit amplitude set to 0). Compute the probabilities at the targets P_j = |ψ_j|².

Form the incoherent sum P_incoherent = Σ_j P_j. If at any target P_all(target) differs from P_incoherent(target) (beyond a small tolerance), that target exhibits interference (constructive or destructive) because cross-terms are present in the squared magnitude.

Notes / assumptions

Vertices indexing: 0 = gun, 1..s = slits, s+1..s+m = targets (targets are absorbing: once amplitude reaches a target it stays with amplitude preserved).

The program uses Python complex arithmetic.

It prints matrices (B and B^2), complex amplitudes, probabilities, the incoherent sum, and highlights targets with interference.

You can enter amplitudes as Python complex literals: 1, 0.5, 1/np.sqrt(2) is not parsed — use 0.70710678 or 0.7071+0j or 0.70710678+0j or 0.70710678j (the parser accepts forms like 1+0j, 0.5-0.2j, -1j).

Short example to try 

If you want the quantum-like interference of the simple double-slit from the book, try:

s = 2, m = 3

gun->slits: 0.70710678 0.70710678 (equal amplitude to each slit, normalized)

slit1 -> targets: 0.577350269 0.577350269 0.577350269 (equal amplitudes to the three targets; these numbers ≈ 1/√3)

slit2 -> targets: 0.577350269 -0.577350269 0.577350269 (introduce a relative sign at the middle target to produce destructive interference)

This will show P_all differing from the incoherent sum at the middle target, demonstrating interference.

In [1]:
#!/usr/bin/env python3
"""
Programming Drill 3.3.2
Multislit experiment with complex amplitudes and interference detection.
"""

from typing import List
import math
import cmath

EPS = 1e-12  # tolerance for floating comparisons

def parse_complex(s: str) -> complex:
    """Parse a complex number from user input. Accepts Python-like 'a+bj' or 'a' or 'bj'."""
    s = s.strip()
    try:
        # allow user to use plain fractions like '1/2' -> evaluate safely:
        if "/" in s and "j" not in s:
            # simple rational -> convert to float then to complex
            num = eval(s, {"__builtins__": None}, {})  # safe limited eval for rational like 1/2
            return complex(float(num))
        return complex(s.replace(" ", ""))
    except Exception:
        raise ValueError(f"Cannot parse '{s}' as a complex number. Examples: 1, 0.5, 1+0j, -0.3j, 0.7071+0.7071j")

def read_int(prompt: str) -> int:
    while True:
        try:
            v = int(input(prompt).strip())
            if v <= 0:
                print("Enter a positive integer.")
                continue
            return v
        except Exception:
            print("Please enter a valid integer.")

def read_complex_vector(n: int, prompt: str) -> List[complex]:
    while True:
        line = input(prompt).strip()
        parts = line.split()
        if len(parts) != n:
            print(f"Expected {n} entries, got {len(parts)}. Try again.")
            continue
        try:
            vec = [parse_complex(p) for p in parts]
            return vec
        except ValueError as e:
            print(e)

def build_B(s: int, m: int,
            gun_to_slits: List[complex],
            slit_to_targets: List[List[complex]]) -> List[List[complex]]:
    """Build B matrix (1+s+m) x (1+s+m) as complex matrix.
       Rows = next position, Columns = current position (so state_next = B * state_current).
    """
    n = 1 + s + m
    B = [[0+0j for _ in range(n)] for __ in range(n)]

    # gun column (col 0) -> slits 1..s
    for i in range(s):
        B[1 + i][0] = gun_to_slits[i]

    # each slit column j -> targets rows s+1 .. s+m
    for j in range(s):
        col = 1 + j
        for t in range(m):
            row = 1 + s + t
            B[row][col] = slit_to_targets[j][t]

    # targets absorb: B[target][target] = 1
    for t in range(m):
        idx = 1 + s + t
        B[idx][idx] = 1 + 0j

    return B

def mat_mult(A: List[List[complex]], B: List[List[complex]]) -> List[List[complex]]:
    p = len(A)
    q = len(A[0]) if p>0 else 0
    assert q == len(B), "Incompatible dims for multiplication"
    r = len(B[0]) if len(B)>0 else 0
    C = [[0+0j for _ in range(r)] for __ in range(p)]
    for i in range(p):
        for j in range(r):
            s = 0+0j
            for k in range(q):
                s += A[i][k] * B[k][j]
            C[i][j] = s
    return C

def mat_vec(A: List[List[complex]], v: List[complex]) -> List[complex]:
    m = len(A)
    n = len(A[0]) if m>0 else 0
    assert n == len(v), "Incompatible dims for mat-vec"
    res = [0+0j for _ in range(m)]
    for i in range(m):
        s = 0+0j
        for j in range(n):
            s += A[i][j] * v[j]
        res[i] = s
    return res

def pretty_complex(z: complex, fmt_real=6, fmt_imag=6) -> str:
    r = round(z.real, fmt_real)
    im = round(z.imag, fmt_imag)
    if abs(im) < 10**(-fmt_imag):
        return f"{r}"
    if abs(r) < 10**(-fmt_real):
        return f"{im}j"
    sign = '+' if im >= 0 else '-'
    return f"{r}{sign}{abs(im)}j"

def print_matrix(A: List[List[complex]], name: str = "Matrix"):
    rows = len(A); cols = len(A[0]) if rows>0 else 0
    print(f"\n{name} ({rows}x{cols}):")
    for row in A:
        print("  " + "  ".join(pretty_complex(x) for x in row))
    print()

def print_vector(v: List[complex], name: str = "Vector"):
    print(f"\n{name}:")
    print("  " + "  ".join(pretty_complex(x) for x in v))
    print()

def amplitudes_to_probabilities(amps: List[complex]) -> List[float]:
    return [ (abs(a)**2) for a in amps ]

def sum_vector_list(vecs: List[List[float]]) -> List[float]:
    if not vecs:
        return []
    n = len(vecs[0])
    s = [0.0]*n
    for v in vecs:
        for i in range(n):
            s[i] += v[i]
    return s

def main():
    print("Programming Drill 3.3.2 — Multislit complex amplitudes & interference detection\n")
    s = read_int("Enter number of slits (s): ")
    m = read_int("Enter number of targets (m): ")

    print(f"\nEnter {s} complex amplitudes for gun -> each slit (space-separated).")
    print("Examples: 1, 0.7071, 0.7071+0j, 0.5+0.5j, -0.3j")
    gun_to_slits = read_complex_vector(s, f"gun->slits ({s} entries): ")

    slit_to_targets = []
    for j in range(s):
        print(f"\nEnter {m} complex amplitudes for slit {j+1} -> each of the {m} targets (space-separated).")
        vec = read_complex_vector(m, f"slit {j+1} -> targets ({m} entries): ")
        slit_to_targets.append(vec)

    # Build B
    B = build_B(s, m, gun_to_slits, slit_to_targets)
    print_matrix(B, "B")

    # Optionally warn if norms not sensible
    # total prob leaving gun (should be 1 in normalized quantum source)
    total_from_gun = sum(abs(a)**2 for a in gun_to_slits)
    print(f"Total probability (|amplitude|^2) leaving gun: {total_from_gun:.12g}")
    if abs(total_from_gun - 1.0) > 1e-9:
        print("  (Warning: gun amplitudes are not normalized to 1 — that's allowed but may be physically inconsistent.)")

    # Construct B^2
    B2 = mat_mult(B, B)
    print_matrix(B2, "B^2")

    # initial state: amplitude 1 at gun
    n = 1 + s + m
    v0 = [0+0j]*n
    v0[0] = 1+0j
    print_vector(v0, "v0 (initial: amplitude 1 at gun)")

    # amplitude distribution after two clicks with all slits open
    v2_all = mat_vec(B2, v0)
    # we want only the target components (indices s+1 ... s+m)
    target_indices = list(range(1 + s, 1 + s + m))
    amps_all_targets = [ v2_all[idx] for idx in target_indices ]
    probs_all_targets = amplitudes_to_probabilities(amps_all_targets)

    print("\nAmplitudes at targets (all slits open) and probabilities:")
    for t in range(m):
        a = amps_all_targets[t]; p = probs_all_targets[t]
        print(f" target {t+1} (vertex {target_indices[t]}): amp = {pretty_complex(a)}, prob = {p:.12g}")

    # Now compute single-slit (only slit j open) scenarios, collect probabilities
    probs_each_slit = []  # list of length s of probability-vectors (length m)
    for j in range(s):
        # build B_j: same as B but gun column only has amplitude for slit j
        gun_vec_j = [0+0j]*s
        gun_vec_j[j] = gun_to_slits[j]
        B_j = build_B(s, m, gun_vec_j, slit_to_targets)
        B_j2 = mat_mult(B_j, B_j)
        v2_j = mat_vec(B_j2, v0)
        amps_j_targets = [ v2_j[idx] for idx in target_indices ]
        probs_j = amplitudes_to_probabilities(amps_j_targets)
        probs_each_slit.append(probs_j)

    # incoherent sum of single-slit probabilities
    probs_incoherent = sum_vector_list(probs_each_slit)

    print("\nIncoherent sum of probabilities from single-slit runs (Σ_j P_j):")
    for t in range(m):
        print(f" target {t+1}: P_incoherent = {probs_incoherent[t]:.12g}")

    # compare coherent vs incoherent -> interference if different
    print("\nComparison (coherent P_all  vs incoherent Σ P_j ) and interference detection:")
    interference_found = False
    for t in range(m):
        p_all = probs_all_targets[t]
        p_inc = probs_incoherent[t]
        diff = p_all - p_inc
        rel = diff if abs(diff) > EPS else 0.0
        sign = "≈" if abs(diff) <= 1e-9 else ("+" if diff > 0 else "-")
        flag = ""
        if abs(diff) > 1e-9:
            interference_found = True
            if diff > 0:
                flag = " (constructive interference at this target)"
            else:
                flag = " (destructive interference at this target)"
        print(f" target {t+1}: P_all = {p_all:.12g}, ΣP_single = {p_inc:.12g}, Δ = {diff:.12g}{flag}")

    if not interference_found:
        print("\nNo interference detected: for every target P_all == Σ P_single (within tolerance).")
    else:
        print("\nInterference detected at targets indicated above (Δ != 0).")

    # show total probability (sanity)
    total_prob_all = sum(probs_all_targets)
    total_prob_incoherent = sum(probs_incoherent)
    print(f"\nTotal prob on targets (coherent): {total_prob_all:.12g}")
    print(f"Total prob on targets (sum single-slit): {total_prob_incoherent:.12g}")

if __name__ == "__main__":
    main()


Programming Drill 3.3.2 — Multislit complex amplitudes & interference detection


Enter 2 complex amplitudes for gun -> each slit (space-separated).
Examples: 1, 0.7071, 0.7071+0j, 0.5+0.5j, -0.3j

Enter 3 complex amplitudes for slit 1 -> each of the 3 targets (space-separated).

Enter 3 complex amplitudes for slit 2 -> each of the 3 targets (space-separated).

B (6x6):
  0.0  0.0  0.0  0.0  0.0  0.0
  0.707107  0.0  0.0  0.0  0.0  0.0
  0.707107  0.0  0.0  0.0  0.0  0.0
  0.0  0.57735  0.57735  1.0  0.0  0.0
  0.0  0.57735  -0.57735  0.0  1.0  0.0
  0.0  0.57735  0.57735  0.0  0.0  1.0

Total probability (|amplitude|^2) leaving gun: 0.999999996644
  (Warning: gun amplitudes are not normalized to 1 — that's allowed but may be physically inconsistent.)

B^2 (6x6):
  0.0  0.0  0.0  0.0  0.0  0.0
  0.0  0.0  0.0  0.0  0.0  0.0
  0.0  0.0  0.0  0.0  0.0  0.0
  0.816497  0.57735  0.57735  1.0  0.0  0.0
  0.0  0.57735  -0.57735  0.0  1.0  0.0
  0.816497  0.57735  0.57735  0.0  0.0  1.0


v0 